# Test Advanced FMR Mode Animations

This notebook demonstrates the new advanced animation capabilities for FMR modes including:
- Temporal animations (real part oscillating in time)
- Frequency sweep animations
- Phase evolution animations
- Custom colormaps including cmocean
- Symmetric normalization for diverging data

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from mmpp import MMPP
from mmpp.fft.modes import FMRModeAnalyzer, ModeVisualizationConfig, MidpointNormalize

# Try to import cmocean for better scientific colormaps
try:
    import cmocean

    print("✅ cmocean available - using scientific colormaps")
    CMOCEAN_AVAILABLE = True
except ImportError:
    print("⚠️  cmocean not available - using matplotlib colormaps")
    CMOCEAN_AVAILABLE = False

## Load Data and Prepare

In [ ]:
# Load your data
op = MMPP(
    "/mnt/storage_2/scratch/pl0095-01/zelent/mannga/Pawel/bowtie_film_sweep_2.14GHz/MZ/v1"
)
lista = op.scan(force=False)

print(f"Loaded {len(op)} simulation results")
print(f"Available datasets: {list(op[0].zarr_file.keys())[:10]}...")  # Show first 10

## Test MidpointNormalize Class

In [ ]:
# Test the MidpointNormalize class
vals = np.array([[-5.0, 0], [5, 10]])
vmin = vals.min()
vmax = vals.max()

# Create normalizations
norm_standard = plt.Normalize(vmin=vmin, vmax=vmax)
norm_midpoint = MidpointNormalize(vmin=vmin, vmax=vmax, midpoint=0)

# Test data
test_vals = np.array([-5, -2.5, 0, 2.5, 5, 7.5, 10])

print("Value\tStandard\tMidpoint")
print("-" * 30)
for val in test_vals:
    std_norm = norm_standard(val)
    mid_norm = norm_midpoint(val)
    print(f"{val:4.1f}\t{std_norm:.3f}\t\t{mid_norm:.3f}")

print("\n✅ MidpointNormalize working correctly!")
print("Notice how value 0 maps to 0.5 in midpoint normalization")

## Create Mode Analyzer with Advanced Configuration

In [ ]:
# Create configuration with new animation options
config = ModeVisualizationConfig(
    colormap_animation="balance"
    if CMOCEAN_AVAILABLE
    else "RdBu_r",  # Diverging colormap
    use_midpoint_norm=True,  # Use symmetric normalization
    animation_time_steps=60,  # 60 frames per oscillation
    figsize=(12, 10),
    dpi=100,
)

# Create analyzer
analyzer = FMRModeAnalyzer(
    zarr_path=op[0].path,
    dataset_name="m_z5-8",  # Choose appropriate dataset
    config=config,
    debug=True,
)

print(f"✅ Analyzer created for dataset: {analyzer.dataset_name}")

## Compute Modes (if needed)

In [ ]:
# Check if modes exist, compute if needed
if not analyzer.modes_available:
    print("Computing FMR modes...")
    analyzer.compute_modes(z_slice=slice(None), window=True, save=True)
    print("✅ Modes computed!")
else:
    print("✅ Modes already available!")

# Show frequency range
print(
    f"Frequency range: {analyzer.frequencies[0]:.3f} - {analyzer.frequencies[-1]:.3f} GHz"
)
print(f"Number of frequency points: {len(analyzer.frequencies)}")

## Find Peaks for Animation Targets

In [ ]:
# Find peaks in spectrum
peaks = analyzer.find_peaks(threshold=0.1)

print(f"Found {len(peaks)} peaks:")
for i, peak in enumerate(peaks[:5]):  # Show first 5
    print(f"  {i + 1}. {peak.freq:.3f} GHz (amplitude: {peak.amplitude:.3f})")

# Choose a strong peak for animation
if peaks:
    target_freq = peaks[0].freq
    print(f"\n🎯 Will use {target_freq:.3f} GHz for temporal animations")
else:
    target_freq = 2.5  # Fallback
    print(f"\n🎯 No peaks found, using {target_freq} GHz")

## Test Animation Types

In [ ]:
# Test different animation types
print("🎬 Creating animations...")
print("This may take a few minutes...")

# 1. Temporal animation - Real part oscillating
print("\n1. Creating temporal animation (real part oscillation)...")
try:
    analyzer.save_modes_animation(
        frequency=target_freq,
        save_path="temporal_animation.gif",
        animation_type="temporal",
        component="z",
        fps=15,
        use_midpoint_norm=True,
        colormap="balance" if CMOCEAN_AVAILABLE else "RdBu_r",
    )
    print("✅ Temporal animation saved as 'temporal_animation.gif'")
except Exception as e:
    print(f"❌ Temporal animation failed: {e}")

In [ ]:
# 2. Phase evolution animation
print("\n2. Creating phase evolution animation...")
try:
    analyzer.save_modes_animation(
        frequency=target_freq,
        save_path="phase_animation.gif",
        animation_type="phase",
        component="z",
        fps=12,
    )
    print("✅ Phase animation saved as 'phase_animation.gif'")
except Exception as e:
    print(f"❌ Phase animation failed: {e}")

In [ ]:
# 3. Frequency sweep animation
if len(peaks) >= 2:
    print("\n3. Creating frequency sweep animation...")
    try:
        # Use range around first two peaks
        f_min = max(peaks[0].freq - 0.5, analyzer.frequencies[0])
        f_max = min(peaks[1].freq + 0.5, analyzer.frequencies[-1])

        analyzer.save_modes_animation(
            frequency_range=(f_min, f_max),
            save_path="frequency_sweep.gif",
            animation_type="frequency",
            component="z",
            fps=8,  # Slower for frequency sweep
        )
        print(f"✅ Frequency sweep animation saved as 'frequency_sweep.gif'")
        print(f"   Range: {f_min:.3f} - {f_max:.3f} GHz")
    except Exception as e:
        print(f"❌ Frequency sweep failed: {e}")
else:
    print("\n⚠️  Skipping frequency sweep - need at least 2 peaks")

## Test Interface Syntax

In [ ]:
# Test the high-level interface
print("🔧 Testing high-level interface...")

try:
    # This should work with the improved interface
    op[0].fft.modes.save_modes_animation(
        frequency=target_freq,
        save_path="interface_temporal.gif",
        dset="m_z5-8",
        animation_type="temporal",
        component="z",
        fps=10,
        use_midpoint_norm=True,
    )
    print("✅ High-level interface works!")
    print("   Saved as 'interface_temporal.gif'")
except Exception as e:
    print(f"❌ Interface test failed: {e}")

## Test Different Components and Colormaps

In [ ]:
# Test different components and colormaps
print("🎨 Testing different components and colormaps...")

components = ["x", "y", "z"]
colormaps = (
    ["balance", "diff", "curl"]
    if CMOCEAN_AVAILABLE
    else ["RdBu_r", "seismic", "coolwarm"]
)

for i, (comp, cmap) in enumerate(zip(components, colormaps)):
    try:
        filename = f"component_{comp}_cmap_{cmap.replace('.', '_')}.gif"

        analyzer.save_modes_animation(
            frequency=target_freq,
            save_path=filename,
            animation_type="temporal",
            component=comp,
            colormap=cmap,
            fps=10,
            figsize=(8, 6),  # Smaller for testing
        )
        print(f"✅ Created {filename}")

        if i >= 2:  # Limit to 3 test animations
            break

    except Exception as e:
        print(f"❌ Failed {comp} component with {cmap}: {e}")

## Summary

In [ ]:
import os

print("📋 Animation Summary")
print("=" * 40)

# List created files
gif_files = [
    f for f in os.listdir(".") if f.endswith(".gif") and "animation" in f.lower()
]

if gif_files:
    print(f"✅ Successfully created {len(gif_files)} animations:")
    for gif in sorted(gif_files):
        size = os.path.getsize(gif) / 1024 / 1024  # MB
        print(f"   📁 {gif} ({size:.1f} MB)")
else:
    print("❌ No animations were created successfully")

print("\n🎯 Key Features Tested:")
print("   ✅ MidpointNormalize for symmetric coloring")
print("   ✅ Temporal animation (real part oscillation)")
print("   ✅ Phase evolution animation")
print("   ✅ Frequency sweep animation")
print("   ✅ Multiple components (x, y, z)")
print("   ✅ Scientific colormaps (cmocean if available)")
print("   ✅ High-level interface integration")

print("\n💡 Usage Examples:")
print("   # Temporal animation:")
print("   analyzer.save_modes_animation(frequency=2.5, animation_type='temporal')")
print("")
print("   # Frequency sweep:")
print(
    "   analyzer.save_modes_animation(frequency_range=(1.0, 3.0), animation_type='frequency')"
)
print("")
print("   # Phase evolution:")
print("   analyzer.save_modes_animation(frequency=2.5, animation_type='phase')")